In [4]:
from simple_coder import SimpleCoder

agenda = {
    "requirements": "I need a script that gets and prints the current weather in Sydney",
    "output_file_name": "weather.sh",
    "working_dir": "~/temp/simple_coder/"
}

coder = SimpleCoder(**agenda)
await coder.run()

Epoch: 0
Epoch: 1


'weather.sh'